<a href="https://colab.research.google.com/github/WillianXG/Projeto_Em_grupo-Modulo-3/blob/main/grupoModulo3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dados Criptomoedas

In [9]:
import pandas as pd
import requests
from matplotlib import pyplot as plt
import seaborn as sns

aave = pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Aave.csv')
binance=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_BinanceCoin.csv')
bitcoin=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Bitcoin.csv')
cardano=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Cardano.csv')
chainlink=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_ChainLink.csv')
cosmos=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Cosmos.csv')
cryptocom=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_CryptocomCoin.csv')
dogecoin=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Dogecoin.csv')
eos=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_EOS.csv')
ethereum=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Ethereum.csv')
iota=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Iota.csv')
litecoin=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Litecoin.csv')
monero=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Monero.csv')
nem=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_NEM.csv')
polkadot=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Polkadot.csv')
solana=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Solana.csv')
stellar=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Stellar.csv')
tether=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Tether.csv')
tron=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Tron.csv')
usd=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_USDCoin.csv')
uniswap=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Uniswap.csv')
wrappedBitcoin=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_WrappedBitcoin.csv')
xrp=pd.read_csv('https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_XRP.csv')
criptomoedas=[aave, binance, bitcoin, cardano, chainlink, cosmos, cryptocom, dogecoin, eos, ethereum, iota, litecoin, monero, nem, polkadot, solana, stellar, tether, tron, usd, uniswap, wrappedBitcoin, xrp]




# Analise de Dados


In [ ]:
criptomoedaMedia=[]
for criptomoeda in criptomoedas:
  # Corrigindo possíveis
  # 1. Verificar valores ausentes
  # valoresEncontrados = criptomoeda.isnull().sum()
  # Exibindo valores nulos
  # print("\nValores Nulos:\n", valoresEncontrados[valoresEncontrados > 0]) if valoresEncontrados.any() else print("Nenhum valor nulo encontrado.")
  # 2. Verificar erros de tipo de dados na coluna 'Date'
  # tipo = pd.DataFrame(criptomoeda.dtypes)
  #   # Exibindo tipos antes da correção
  # print(tipo)
  # tipo = pd.DataFrame(criptomoeda.dtypes)
  # criptomoeda = criptomoeda.astype({col: 'string' for col in criptomoeda.select_dtypes(include='object').columns})
  # # Exibindo tipos pós correção
  # print('\n\n\n')
  # print(criptomoeda['Name'][0])
  # tipo = pd.DataFrame(criptomoeda.dtypes)
  # print(tipo)
  # print('\n\n\n')
  #3. Verificar duplicatas
  duplicatas = criptomoeda.duplicated().sum()
  print(duplicatas)
  if duplicatas > 0:
       criptomoeda=criptomoeda.drop_duplicates()
       print("\nDuplicate Rows Found: ", duplicatas)
  else: print("Sem Colunas duplicadas.")
  duplicatas = criptomoeda.duplicated().sum()
  print(f'Depois de corrigir \n\n{duplicatas}')
  if duplicatas > 0:
       print("\nDuplicate Rows Found: ", duplicatas)
  else: print("Sem Colunas duplicadas.")

  # # 4. (Opcional) Verificar outliers na coluna 'Close'
  # if 'Close' in criptomoeda.columns:
  #     q1 = criptomoeda['Close'].quantile(0.25)
  #     q3 = criptomoeda['Close'].quantile(0.75)
  #     iqr = q3 - q1
  #     lower_bound = q1 - 1.5 * iqr
  #     upper_bound = q3 + 1.5 * iqr
  #     outliers = criptomoeda[(criptomoeda['Close'] < lower_bound) | (criptomoeda['Close'] > upper_bound)]
  #     if not outliers.empty:
  #         print("\nOutliers Found:\n", outliers[['Date', 'Close']]
  # Corrigir os problemas encontrado
  # Remover linhas onde a data não pôde ser convertida
  #criptomoeda = criptomoeda.dropna(subset=['Date']
  # Preencher valores ausentes

    # Calculando valores das perguntas
        #1. Como se comportaram os valores para todas as criptomoedas? Os valores tiveram uma tendência de queda ou de aumento?
        #2. Quais os valores médios para todas as criptomoedas?
        #3. Em quais anos houve maiores quedas e valorizações?
        #4. Existe alguma tendência de aumento ou queda dos valores pelo dia da semana?
        #5. Qual moeda se mostra mais interessante em relação à valorização pela análise da série histórica?
        #6. Qual moeda se mostra menos interessante em relação à valorização pela análise da série histórica?
        #7. Existe correlação entre os valores para todas as criptomoedas?
        #8.
        #9.
    # Repondendo as perguntas
        #1. Como se comportaram os valores para todas as criptomoedas? Os valores tiveram uma tendência de queda ou de aumento?
        #2. Quais os valores médios para todas as criptomoedas?
print(criptomoedaMedia)
        #3. Em quais anos houve maiores quedas e valorizações?
        #4. Existe alguma tendência de aumento ou queda dos valores pelo dia da semana?
        #5. Qual moeda se mostra mais interessante em relação à valorização pela análise da série histórica?
        #6. Qual moeda se mostra menos interessante em relação à valorização pela análise da série histórica?
        #7. Existe correlação entre os valores para todas as criptomoedas?
        #8.
        #9.

0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Sem Colunas duplicadas.
0
Sem Colunas duplicadas.
Depois de corrigir 

0
Se

1. Como se comportaram os valores para todas as criptomoedas? Os valores tiveram uma
tendência de queda ou de aumento?
2. Quais os valores médios para todas as criptomoedas?

# JUNTANDO TODOS OS CSVs EM APNES UM.


#listando o novo csv
csv_unico  = [
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Aave.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_BinanceCoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Bitcoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Cardano.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_ChainLink.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Cosmos.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_CryptocomCoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Dogecoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_EOS.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Ethereum.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Iota.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Litecoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Monero.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_NEM.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Polkadot.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Solana.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Stellar.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Tether.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Tron.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_USDCoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_Uniswap.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_WrappedBitcoin.csv',
    'https://raw.githubusercontent.com/davidlgomes/modulo3AnaliseDados/main/coin_XRP.csv'
]


li = []  #lista para armazenar o arquivo csv novo


# Gerando um DataFrame com uma função em loop

for url in csv_unico:
    df = pd.read_csv(url)
    li.append(df)

df_unico = pd.concat(li, axis=0, ignore_index=True)    # Realizando a concatenização
df_unico


3. Em quais anos houve maiores quedas e valorizações?
4. Existe alguma tendência de aumento ou queda dos valores pelo dia da semana?
5. Qual moeda se mostra mais interessante em relação à valorização pela análise da série
histórica?
6. Qual moeda se mostra menos interessante em relação à valorização pela análise da série
histórica?
7. Existe correlação entre os valores para todas as criptomoedas?